# 0. Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Dacon_RF_parameter_tuning

/content/drive/MyDrive/Dacon_RF_parameter_tuning


In [ ]:
ls

 0325_EDA_Gridsearch.ipynb         grid_test2.ipynb        submit0329_5.csv
 0326_Generalization.ipynb         sample_submission.csv   submit.ipynb
 0327_RandomSearch.ipynb           submit/                 test_grid.ipynb
 0328_submit.ipynb                 submit0329_2.csv       'test_grid.ipynb의 사본'
 0329_BayesianOptimization.ipynb   submit0329_3.csv        train.csv
 EDA.ipynb                         submit0329_4.csv


In [ ]:
# 대회 제공 시드로 고정
import random
random.seed(42)

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame

In [ ]:
train = pd.read_csv('train.csv')
train.head()

,person_id,Sex,past_login_total,past_1_month_login,past_1_week_login,sub_size,email_type,phone_rat,apple_rat,login
0,0,0,3.0,1.0,1.0,0.0,0,0.000000,0.000000,1
1,1,1,111.0,26.0,7.0,2.0,0,0.072072,0.000000,1
2,3,1,13.0,13.0,11.0,7.0,0,0.076923,1.000000,1
3,4,1,28.0,12.0,5.0,0.0,0,0.071429,0.071429,1
4,5,1,4.0,4.0,4.0,0.0,2,0.000000,0.000000,1


# 1. Bayesian Optimization

참고 : https://www.kaggle.com/code/neerajmohan/randomforest-model-with-bayesian-optimization

In [ ]:
!pip install bayesian-optimization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report,precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from bayes_opt import BayesianOptimization

In [ ]:
from bayes_opt import BayesianOptimization

data split

In [ ]:
X = train.drop(labels='login',axis=1)
X
y = train['login']
y

0       1
1       1
2       1
3       1
4       1
       ..
1304    0
1305    0
1306    0
1307    0
1308    0
Name: login, Length: 1309, dtype: int64

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

model

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.fit(X_train,y_train)


RandomForestClassifier()

In [ ]:
preds = rf.predict(X_test)

In [ ]:
print(classification_report(preds,y_test)) # class 1 에 대한 precision, recall, f1-score, support가 전체적으로 낮음 (데이터 편향)

              precision    recall  f1-score   support

           0       0.97      0.92      0.95       369
           1       0.34      0.62      0.44        24

    accuracy                           0.90       393
   macro avg       0.66      0.77      0.69       393
weighted avg       0.94      0.90      0.92       393



In [ ]:
print("Precision:{}".format(precision_score(preds,y_test)))
print("Recall:{}".format(recall_score(preds,y_test)))
print("F1 Score:{}".format((f1_score(preds,y_test))))

Precision:0.3409090909090909
Recall:0.625
F1 Score:0.4411764705882353


In [ ]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# 필요한 패키지 설치"
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 1.1 MB/s eta 0:00:00


In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer


In [ ]:
# try1 : 몇 파라미터에 대해서만 Bayesian Optimization test -> 결과 잘 나옴, 실행 시간 grid search와 random search에 비해 매우 빠름
# 특성과 종속변수 분리
X = train.drop('login', axis=1)
y = train['login']

# 랜덤 포레스트 모델 정의
rf = RandomForestClassifier()

# 탐색할 하이퍼파라미터 공간 정의
param_space = {
    'n_estimators': Integer(10, 200),  # 나무의 수
    'max_depth': Integer(1, 100),       # 트리의 최대 깊이
    'min_samples_split': Integer(2, 20),# 내부 노드를 분할하는데 필요한 최소 샘플 수
    'min_samples_leaf': Integer(1, 20), # 리프 노드에 필요한 최소 샘플 수
    'max_features': Categorical(['auto', 'sqrt', 'log2']), # 각 노드에서 분할에 사용할 특성의 최대 수
    'bootstrap': Categorical([True, False]) # 데이터 중복 사용 여부
}

# Bayesian Optimization을 위한 검색 객체 정의
opt = BayesSearchCV(
    estimator=rf,
    search_spaces=param_space,
    n_iter=50, # 반복 횟수
    cv=5,      # 교차 검증 폴드 수
    n_jobs=-1, # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    verbose=0
)

# Bayesian Optimization 실행
opt.fit(X, y)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", opt.best_params_)

# 최적의 하이퍼파라미터를 가진 모델의 성능 출력
best_model = opt.best_estimator_
best_score = opt.best_score_
print("최적의 하이퍼파라미터를 가진 모델의 성능 (평균 교차 검증 정확도):", best_score)

최적의 하이퍼파라미터: OrderedDict([('bootstrap', True), ('max_depth', 1), ('max_features', 'auto'), ('min_samples_leaf', 17), ('min_samples_split', 20), ('n_estimators', 10)])
최적의 하이퍼파라미터를 가진 모델의 성능 (평균 교차 검증 정확도): 0.8701441899915181


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [ ]:
# try2 :파인튜닝하고자하는 모든 하이퍼파라미터에 대해 수행, 성능 출력 -> 결과 잘 나옴
# 특성과 종속변수 분리
X = train.drop('login', axis=1)
y = train['login']

# 랜덤 포레스트 모델 정의
rf = RandomForestClassifier()

# 탐색할 하이퍼파라미터 공간 정의
param_space = {
    'n_estimators': Integer(10, 200),  # 나무의 수, 범위: 10 ~ 1000
    'criterion' : Categorical(['gini', 'entropy']), # 옵션: 'gini', 'entropy'
    'max_depth': Integer(1, 100),       # 트리의 최대 깊이,범위: None 또는 양의 정수 / 현재 - 1이상, 100 이하 정수 범위로 탐색
    'min_samples_split': Integer(2, 20),# 내부 노드를 분할하는데 필요한 최소 샘플 수, 범위: 2 이상의 정수 또는 0과 1 사이의 실수
    'min_samples_leaf': Integer(1, 20), # 리프 노드에 필요한 최소 샘플 수, 범위: 1 이상의 정수 또는 0과 0.5 사이의 실수
    'min_weight_fraction_leaf' :Real(0.0, 0.5),  # 범위: 0.0에서 0.5 사이의 실수
    'max_features': Categorical(['auto', 'sqrt', 'log2']), # 각 노드에서 분할에 사용할 특성의 최대 수, 옵션: 'auto', 'sqrt', 'log2', None 또는 양의 정수/실수
    'max_leaf_nodes' : Integer(2, 100),  # 범위: None 또는 양의 정수
    'min_impurity_decrease' : Real(0.0, 0.5),   # 범위: 0.0 이상의 실수
    'bootstrap': Categorical([True, False]) # 데이터 중복 사용 여부, 옵션: True, False
}

# Bayesian Optimization을 위한 검색 객체 정의
opt = BayesSearchCV(
    estimator=rf,
    search_spaces=param_space,
    n_iter=50, # 반복 횟수
    cv=5,      # 교차 검증 폴드 수
    n_jobs=-1, # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    verbose=0
)

# Bayesian Optimization 실행
opt.fit(X, y)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", opt.best_params_)

# 최적의 하이퍼파라미터를 가진 모델의 성능 출력
best_model = opt.best_estimator_
best_score = opt.best_score_
print("최적의 하이퍼파라미터를 가진 모델의 성능 (평균 교차 검증 정확도):", best_score)

최적의 하이퍼파라미터: OrderedDict([('bootstrap', False), ('criterion', 'entropy'), ('max_depth', 16), ('max_features', 'sqrt'), ('max_leaf_nodes', 84), ('min_impurity_decrease', 0.13407446029425837), ('min_samples_leaf', 4), ('min_samples_split', 15), ('min_weight_fraction_leaf', 0.31663831795626113), ('n_estimators', 30)])
최적의 하이퍼파라미터를 가진 모델의 성능 (평균 교차 검증 정확도): 0.8884648006785412


In [ ]:
# try3 :탐색 범위 늘리기(대회에서 정한 모든 범위에 대하여) - 많은 범위임에도 gridsearch, random search에 비해 확연히 빠름 (10분 내 실행)
# 특성과 종속변수 분리
X = train.drop('login', axis=1)
y = train['login']

# 랜덤 포레스트 모델 정의
rf = RandomForestClassifier()

# 탐색할 하이퍼파라미터 공간 정의
param_space = {
    'n_estimators': Integer(10, 1000),  # 나무의 수, 범위: 10 ~ 1000
    'criterion' : Categorical(['gini', 'entropy']), # 옵션: 'gini', 'entropy'
    'max_depth': Integer(1, 100),       # 트리의 최대 깊이,범위: None 또는 양의 정수 / 현재 - 1이상, 100 이하 정수 범위로 탐색
    'min_samples_split': Integer(2, 30),# 내부 노드를 분할하는데 필요한 최소 샘플 수, 범위: 2 이상의 정수 또는 0과 1 사이의 실수
    'min_samples_leaf': Integer(1, 30), # 리프 노드에 필요한 최소 샘플 수, 범위: 1 이상의 정수 또는 0과 0.5 사이의 실수
    'min_weight_fraction_leaf' :Real(0.0, 0.5),  # 범위: 0.0에서 0.5 사이의 실수
    'max_features': Categorical(['auto', 'sqrt', 'log2', None]), # 각 노드에서 분할에 사용할 특성의 최대 수, 옵션: 'auto', 'sqrt', 'log2', None 또는 양의 정수/실수
    'max_leaf_nodes' : Integer(2, 120),  # 범위: None 또는 양의 정수
    'min_impurity_decrease' : Real(0.0, 0.5),   # 범위: 0.0 이상의 실수
    'bootstrap': Categorical([True, False]) # 데이터 중복 사용 여부, 옵션: True, False
}

# Bayesian Optimization을 위한 검색 객체 정의
opt = BayesSearchCV(
    estimator=rf,
    search_spaces=param_space,
    n_iter=50, # 반복 횟수
    cv=5,      # 교차 검증 폴드 수
    n_jobs=-1, # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    verbose=0
)

# Bayesian Optimization 실행
opt.fit(X, y)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", opt.best_params_)

# 최적의 하이퍼파라미터를 가진 모델의 성능 출력
best_model = opt.best_estimator_
best_score = opt.best_score_
print("최적의 하이퍼파라미터를 가진 모델의 성능 (평균 교차 검증 정확도):", best_score)

최적의 하이퍼파라미터: OrderedDict([('bootstrap', True), ('criterion', 'gini'), ('max_depth', 15), ('max_features', None), ('max_leaf_nodes', 52), ('min_impurity_decrease', 0.39493616009169324), ('min_samples_leaf', 28), ('min_samples_split', 6), ('min_weight_fraction_leaf', 0.23438516015372562), ('n_estimators', 49)])
최적의 하이퍼파라미터를 가진 모델의 성능 (평균 교차 검증 정확도): 0.8884648006785412


# 2. 데이터의 편향 고려한 Bayesian Optimization
- 시도1) balanced
- 시도2) balanced + 언더샘플링 + 평가함수 f1-score로 변경
- 시도3) balanced + f1-score

In [ ]:
# 1. balanced로 편향 문제 해결
# 위 결과의 성능이 너무 낮게 나와, 데이터의 편향 고려하는 베이지안 옵티마이제이션으로 변경
# -> balanced 사용
from sklearn.utils import class_weight

# 특성과 종속변수 분리
X = train.drop('login', axis=1)
y = train['login']

# 랜덤 포레스트 모델 정의
rf = RandomForestClassifier()

# 탐색할 하이퍼파라미터 공간 정의
param_space = {
    'n_estimators': Integer(10, 1000),  # 나무의 수, 범위: 10 ~ 1000
    'criterion' : Categorical(['gini', 'entropy']), # 옵션: 'gini', 'entropy'
    'max_depth': Integer(1, 100),       # 트리의 최대 깊이,범위: None 또는 양의 정수 / 현재 - 1이상, 100 이하 정수 범위로 탐색
    'min_samples_split': Integer(2, 30),# 내부 노드를 분할하는데 필요한 최소 샘플 수, 범위: 2 이상의 정수 또는 0과 1 사이의 실수
    'min_samples_leaf': Integer(1, 30), # 리프 노드에 필요한 최소 샘플 수, 범위: 1 이상의 정수 또는 0과 0.5 사이의 실수
    'min_weight_fraction_leaf' :Real(0.0, 0.5),  # 범위: 0.0에서 0.5 사이의 실수
    'max_features': Categorical(['auto', 'sqrt', 'log2', None]), # 각 노드에서 분할에 사용할 특성의 최대 수, 옵션: 'auto', 'sqrt', 'log2', None 또는 양의 정수/실수
    'max_leaf_nodes' : Integer(2, 120),  # 범위: None 또는 양의 정수
    'min_impurity_decrease' : Real(0.0, 0.5),   # 범위: 0.0 이상의 실수
    'bootstrap': Categorical([True, False]) # 데이터 중복 사용 여부, 옵션: True, False
}
# 클래스 가중치 계산 - 데이터의 편향 고려 -> balanced
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)

# Bayesian Optimization을 위한 검색 객체 정의
opt = BayesSearchCV(
    estimator=RandomForestClassifier(class_weight='balanced'),
    search_spaces=param_space,
    n_iter=50, # 반복 횟수
    cv=5,      # 교차 검증 폴드 수
    n_jobs=-1, # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    verbose=0
)

# Bayesian Optimization 실행
opt.fit(X, y)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", opt.best_params_)

# 최적의 하이퍼파라미터를 가진 모델의 성능 출력
best_model = opt.best_estimator_
best_score = opt.best_score_
print("최적의 하이퍼파라미터를 가진 모델의 성능 (평균 교차 검증 정확도):", best_score)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


최적의 하이퍼파라미터: OrderedDict([('bootstrap', True), ('criterion', 'gini'), ('max_depth', 84), ('max_features', 'auto'), ('max_leaf_nodes', 45), ('min_impurity_decrease', 0.34277318276007895), ('min_samples_leaf', 7), ('min_samples_split', 3), ('min_weight_fraction_leaf', 0.22047399965168385), ('n_estimators', 868)])
최적의 하이퍼파라미터를 가진 모델의 성능 (평균 교차 검증 정확도): 0.8884648006785412


In [ ]:
# 3. balanced + 언더샘플링 + f1-score로 데이터 편향 해결
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight

# 언더샘플링을 통해 클래스 불균형 해결
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

# 훈련 및 테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# 클래스 가중치 계산
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# F1 점수를 사용하여 모델 평가
scorer = make_scorer(f1_score)

# 탐색할 하이퍼파라미터 공간 정의
param_space = {
    'n_estimators': Integer(10, 1000),                # 나무의 수
    'criterion': Categorical(['gini', 'entropy']),    # 분할 기준
    'max_depth': Integer(1, 100),                     # 트리의 최대 깊이
    'min_samples_split': Integer(2, 30),              # 내부 노드를 분할하는데 필요한 최소 샘플 수
    'min_samples_leaf': Integer(1, 30),               # 리프 노드에 필요한 최소 샘플 수
    'min_weight_fraction_leaf': Real(0.0, 0.5),       # 가중치 고려 최소 리프 노드 크기
    'max_features': Categorical(['auto', 'sqrt', 'log2', None]),  # 각 노드에서 분할에 사용할 특성의 최대 수
    'max_leaf_nodes': Integer(2, 120),                # 리프 노드의 최대 수
    'min_impurity_decrease': Real(0.0, 0.5),          # 분할을 위한 최소 불순도 감소
    'bootstrap': Categorical([True, False])           # 데이터 중복 사용 여부
}

# Bayesian Optimization을 위한 검색 객체 정의
opt = BayesSearchCV(
    estimator=RandomForestClassifier(class_weight=class_weights_dict), # 클래스 가중치 적용
    search_spaces=param_space,
    scoring=scorer,
    n_iter=50, # 반복 횟수
    cv=5,      # 교차 검증 폴드 수
    n_jobs=-1, # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    verbose=0
)

# Bayesian Optimization 실행
opt.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", opt.best_params_)

# 최적의 하이퍼파라미터를 가진 모델의 성능 출력
best_model = opt.best_estimator_
best_score = opt.best_score_
print("최적의 하이퍼파라미터를 가진 모델의 성능 (평균 F1 점수):", best_score)

# 테스트 데이터에서 모델 평가
test_score = f1_score(y_test, best_model.predict(X_test))
print("테스트 데이터에서 모델의 성능 (F1 점수):", test_score)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


최적의 하이퍼파라미터: OrderedDict([('bootstrap', False), ('criterion', 'gini'), ('max_depth', 85), ('max_features', 'auto'), ('max_leaf_nodes', 105), ('min_impurity_decrease', 0.0), ('min_samples_leaf', 12), ('min_samples_split', 3), ('min_weight_fraction_leaf', 0.0), ('n_estimators', 817)])
최적의 하이퍼파라미터를 가진 모델의 성능 (평균 F1 점수): 0.7669351817530334
테스트 데이터에서 모델의 성능 (F1 점수): 0.7407407407407408


In [ ]:
# 3. balanced + f1-score로 데이터 편향 해결

from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight

# 훈련 및 테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 클래스 가중치 계산
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# F1 점수를 사용하여 모델 평가
scorer = make_scorer(f1_score)

# 탐색할 하이퍼파라미터 공간 정의
param_space = {
    'n_estimators': Integer(10, 1000),                # 나무의 수
    'criterion': Categorical(['gini', 'entropy']),    # 분할 기준
    'max_depth': Integer(1, 100),                     # 트리의 최대 깊이
    'min_samples_split': Integer(2, 30),              # 내부 노드를 분할하는데 필요한 최소 샘플 수
    'min_samples_leaf': Integer(1, 30),               # 리프 노드에 필요한 최소 샘플 수
    'min_weight_fraction_leaf': Real(0.0, 0.5),       # 가중치 고려 최소 리프 노드 크기
    'max_features': Categorical(['auto', 'sqrt', 'log2', None]),  # 각 노드에서 분할에 사용할 특성의 최대 수
    'max_leaf_nodes': Integer(2, 120),                # 리프 노드의 최대 수
    'min_impurity_decrease': Real(0.0, 0.5),          # 분할을 위한 최소 불순도 감소
    'bootstrap': Categorical([True, False])           # 데이터 중복 사용 여부
}

# Bayesian Optimization을 위한 검색 객체 정의
opt = BayesSearchCV(
    estimator=RandomForestClassifier(class_weight=class_weights_dict), # 클래스 가중치 적용
    search_spaces=param_space,
    scoring=scorer,
    n_iter=50, # 반복 횟수
    cv=5,      # 교차 검증 폴드 수
    n_jobs=-1, # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    verbose=0
)

# Bayesian Optimization 실행
opt.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", opt.best_params_)

# 최적의 하이퍼파라미터를 가진 모델의 성능 출력
best_model = opt.best_estimator_
best_score = opt.best_score_
print("최적의 하이퍼파라미터를 가진 모델의 성능 (평균 F1 점수):", best_score)

# 테스트 데이터에서 모델 평가
test_score = f1_score(y_test, best_model.predict(X_test))
print("테스트 데이터에서 모델의 성능 (F1 점수):", test_score)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


최적의 하이퍼파라미터: OrderedDict([('bootstrap', True), ('criterion', 'entropy'), ('max_depth', 93), ('max_features', 'auto'), ('max_leaf_nodes', 17), ('min_impurity_decrease', 0.0), ('min_samples_leaf', 30), ('min_samples_split', 23), ('min_weight_fraction_leaf', 0.2929723323941858), ('n_estimators', 520)])
최적의 하이퍼파라미터를 가진 모델의 성능 (평균 F1 점수): 0.5034011331805449
테스트 데이터에서 모델의 성능 (F1 점수): 0.4657534246575342


# 3. 제출

In [ ]:
# 제출1. 편향 고려하지 않은 BayesianOptimization
df = {'n_estimators':49,
      'criterion' : 'gini',
      'max_depth' : 15,
      'min_samples_split' : 6,
      'min_samples_leaf' : 28,
      'min_weight_fraction_leaf' : 0.23438516015372562,
      'max_features' : None,
      'max_leaf_nodes' : 52,
      'min_impurity_decrease' : 0.39493616009169324,
      'bootstrap': True
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0329_3.csv", index=False)

In [ ]:
#제출2. balanced
df = {'n_estimators':868,
      'criterion' : 'gini',
      'max_depth' : 84,
      'min_samples_split' : 3,
      'min_samples_leaf' : 7,
      'min_weight_fraction_leaf' : 0.22047399965168385,
      'max_features' : 'auto',
      'max_leaf_nodes' : 45,
      'min_impurity_decrease' : 0.34277318276007895,
      'bootstrap': True
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0329_4.csv", index=False)

In [ ]:
#제출3. balanced + f1-score
df = {'n_estimators':520,
      'criterion' : 'entropy',
      'max_depth' : 93,
      'min_samples_split' : 23,
      'min_samples_leaf' : 30,
      'min_weight_fraction_leaf' : 0.2929723323941858,
      'max_features' : 'auto',
      'max_leaf_nodes' : 17,
      'min_impurity_decrease' : 0.0,
      'bootstrap': True
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0329_5.csv", index=False)

In [ ]:
# 제출4. balanced + 언더샘플링 + f1-score -> 최고점
df = {'n_estimators':817,
      'criterion' : 'gini',
      'max_depth' : 85,
      'min_samples_split' : 3,
      'min_samples_leaf' : 12,
      'min_weight_fraction_leaf' : 0.0,
      'max_features' : 'auto',
      'max_leaf_nodes' : 105,
      'min_impurity_decrease' : 0.0,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0329_6.csv", index=False)

In [ ]:
# 제출5. 제출4에서 max_depth 5 감소
df = {'n_estimators':817,
      'criterion' : 'gini',
      'max_depth' : 80,
      'min_samples_split' : 3,
      'min_samples_leaf' : 12,
      'min_weight_fraction_leaf' : 0.0,
      'max_features' : 'auto',
      'max_leaf_nodes' : 105,
      'min_impurity_decrease' : 0.0,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0329_7.csv", index=False)

# 4. 제출 결과

- 제출1) 편향 고려하지 않은 Bayesian Optimization
 - score : 0.5
 - 예상보다 너무 낮은 score로, 데이터의 편향된 특성을 반영해야겠다는 아이디어를 얻게 됨
- 제출2) balanced
 - score : 0.5
 - 여전히 낮은 score가 나타나, 다른 방법으로 데이터 특성을 반영해보기로 함
- 제출3) balanced + f1-score  
 - score : 0.6268987342
 - 성능이 향상되었지만, 여전히 이전 최고기록보다 매우 낮은 성능
- 제출4) balanced + 언더샘플링 + f1-score
 - score : 0.8034810127
 - 최고점 기록
- 제출5) 제출4에서 max_depth 5 감소시켜 제
 - score : 0.8034810127
 - 제출4와 동일한 성능
